In [ ]:
import pyreadr

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


# Discovery

### Load Datasets

In [ ]:
# load discovery results
discovery = pyreadr.read_r('results/discovery_2000s_2r_nc.rdata')["discovery"]
discovery.replace("NaN","0", inplace=True)
discovery.iloc[:,1:] = discovery.iloc[:,1:].astype(float)

# filtering
xmax = 4
methods = ["CRE (aipw)", "CRE (cf)", "CRE (bcf)", "HCT"]
discovery = discovery[(discovery.method.isin(methods)) & (discovery.effect_size<=xmax)]
discovery

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(7, 5))
#fig.suptitle('Discovery')
subfigs = fig.subfigures(nrows=1, ncols=2)
labelsize = 8

# Causal Decision Rules
subfigs[0].suptitle(f'Causal Decision Rules', size=10)
ax = subfigs[0].subplots(nrows=3, ncols=1)

sns.lineplot(data=discovery, x="effect_size", y="cdr_Recall", hue="method", ax=ax[0])
ax[0].set_xlabel('Effect Size', size=labelsize)  
ax[0].set_ylabel('Recall', size=labelsize) 
ax[0].set_ylim((0,1)) 
ax[0].set_xlim((0,xmax))
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[0].tick_params(axis='both', which='major', labelsize=labelsize)
ax[0].get_legend().remove()

sns.lineplot(data=discovery, x="effect_size", y="cdr_Precision", hue="method", ax=ax[1])
ax[1].set_xlabel('Effect Size', size=labelsize)  
ax[1].set_ylabel('Precision', size=labelsize) 
ax[1].set_ylim((0,1)) 
ax[1].set_xlim((0,xmax))
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].tick_params(axis='both', which='major', labelsize=labelsize)
ax[1].get_legend().remove()

sns.lineplot(data=discovery, x="effect_size", y="cdr_IoU", hue="method", ax=ax[2])
ax[2].set_xlabel('Effect Size', size=labelsize)  
ax[2].set_ylabel('IoU', size=labelsize)  
ax[2].set_ylim((0,1))
ax[2].set_xlim((0,xmax))
ax[2].spines['top'].set_visible(False)
ax[2].spines['right'].set_visible(False)
ax[2].tick_params(axis='both', which='major', labelsize=labelsize)
ax[2].get_legend().remove()


# Effect Modifiers
subfigs[1].suptitle(f'Effect Modifiers', size=10)
ax = subfigs[1].subplots(nrows=3, ncols=1)

sns.lineplot(data=discovery, x="effect_size", y="em_Recall", hue="method", ax=ax[0])
ax[0].set_xlabel('Effect Size', size=labelsize) 
ax[0].set_ylabel('Recall', size=labelsize) 
ax[0].set_ylim((0,1))
ax[0].set_xlim((0,xmax))
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[0].tick_params(axis='both', which='major', labelsize=labelsize)
ax[0].get_legend().remove()  

sns.lineplot(data=discovery, x="effect_size", y="em_Precision", hue="method", ax=ax[1])
ax[1].set_xlabel('Effect Size', size=labelsize)  
ax[1].set_ylabel('Precision', size=labelsize)  
ax[1].set_ylim((0,1))
ax[1].set_xlim((0,xmax))
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[1].tick_params(axis='both', which='major', labelsize=labelsize)
ax[1].get_legend().remove()

sns.lineplot(data=discovery, x="effect_size", y="em_IoU", hue="method", ax=ax[2])
ax[2].set_xlabel('Effect Size', size=labelsize) 
ax[2].set_ylabel('IoU', size=labelsize)  
ax[2].set_ylim((0,1))
ax[2].set_xlim((0,xmax))
ax[2].spines['top'].set_visible(False)
ax[2].spines['right'].set_visible(False)
ax[2].tick_params(axis='both', which='major', labelsize=labelsize)
ax[2].get_legend().remove()

handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper right', ncol=4, bbox_to_anchor=(0.8, 0), fontsize=labelsize);

# Estimation

### Load Dataset

In [ ]:
# load estimation results
estimation = pyreadr.read_r('results/estimation_5000s_2r_5es_nc.rdata')["estimation"]
estimation.iloc[:,1:] = estimation.iloc[:,1:].astype(float)

# standardize
estimation["beta1"] = (estimation["beta1"]+estimation["effect_size"])/estimation["effect_size"]
estimation["beta2"] = (estimation["beta2"]-estimation["effect_size"])/estimation["effect_size"]
#estimation["beta3"] = (estimation["beta3"]+estimation["effect_size"])/estimation["effect_size"]
#estimation["beta4"] = (estimation["beta4"]-estimation["effect_size"])/estimation["effect_size"]

# filtering
estimation.drop(columns=['beta3', 'beta4'], axis=1, inplace=True)
methods = ["CRE (aipw)", "CRE (cf)", "CRE (bcf)", "CF", "BCF", "AIPW", "HCT"]
estimation = estimation[estimation.method.isin(methods)]

estimation.head()

### ITE

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(9, 3), layout='constrained')
fig.suptitle('Estimation - Individual Treatment Effect')

sns.boxplot(data=estimation, x="rmse", y="method", ax=ax[0])
ax[0].set_xlabel('RMSE')  
ax[0].set_ylabel('Method')
ax[0].set_xlim((0,3))
ax[0].axvline(-5, 0, 5, color="black", linestyle=":"); 

sns.boxplot(data=estimation, x="bias", y="method", ax=ax[1])
ax[1].set_xlabel('Bias')  
ax[1].set_ylabel('Method')
ax[1].set_xlim((-0.4,0.4))
ax[1].axvline(5, 0, 5, color="black", linestyle=":")
ax[1].axvline(0, 0, 5, color="black", linestyle=":");  

### CATE

In [ ]:
estimation_beta = estimation.dropna()
fig, ax = plt.subplots(1,2,figsize=(9, 3), layout='constrained')
fig.suptitle('Estimation - β (CATE)')

sns.boxplot(data=estimation_beta, x="beta1", y="method", ax=ax[0])
ax[0].set_title('Rule 1')
ax[0].set_xlabel('Bias')  
ax[0].set_ylabel('Method')
ax[0].set_xlim((-0.5,0.5))
ax[0].axvline(0, 0, 5, color="black", linestyle=":"); 

sns.boxplot(data=estimation_beta, x="beta2", y="method", ax=ax[1])
ax[1].set_title('Rule 2')
ax[1].set_xlabel('Bias')  
ax[1].set_ylabel('Method')
ax[1].set_xlim((-0.5,0.5))
ax[1].axvline(0, 0, 5, color="black", linestyle=":"); 

### Summary

In [ ]:
estimation_summary = estimation.drop(columns=['effect_size', 'seed'], axis=1,)
estimation_summary = estimation_summary.groupby("method").agg(["mean","std"])
estimation_summary